# NLP: Introdução a regex e modelos de linguagem

Material referente ao curso da Alura disponível [aqui](https://cursos.alura.com.br/course/nlp-modelos-linguagem). O propósito final deste notebook é criar um detector de linguagem utilizando regex e modelos de linguagem.

Serão utilizadas expressões regulares para facilitar o reconhecimento de padrões no texto. Mais informações [aqui](https://regex101.com/).

## Preparando o ambiente

In [1]:
import pandas as pd

## Carregando os dados

In [2]:
dados_portugues = pd.read_csv('dados/stackoverflow_portugues.csv')
dados_ingles = pd.read_csv('dados/stackoverflow_ingles.csv')
dados_espanhol = pd.read_csv('dados/stackoverflow_espanhol.csv')

In [3]:
dados_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191
